### 평가 23.05.19. 2번

- 피마 인디언 당뇨병 데이터세트

In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv('../data/pima-indians-diabetes.csv', skiprows=9, header=None)
df.columns = ['pregnant', 'plasma', 'pressure', 'skin', 'insulin', 'bmi', 'predigree', 'age', 'target']

In [5]:
df.isna().sum()

pregnant     0
plasma       0
pressure     0
skin         0
insulin      0
bmi          0
predigree    0
age          0
target       0
dtype: int64

In [6]:
df.head()

,pregnant,plasma,pressure,skin,insulin,bmi,predigree,age,target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


1. 각 Feature의 최소값이 0, 최대값이 1이 되도록 MinMaxScaler를 사용하여
   변환하세요.

In [7]:
from sklearn.preprocessing import MinMaxScaler
X = df.iloc[:, :-1].values
y = df.target.values

X_mn = MinMaxScaler().fit_transform(X)

2. SVC, KNN, Logistic Regression을 소프트 보팅 방식으로 앙상블 학습을 하되,   SVC의 C 파라메터와 LR의 C 파라메터를 최적화해서 분류를 시도하세요.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_mn, y, stratify=y, test_size=0.2, random_state=2023
)

In [10]:
# SVC
from sklearn.svm import SVC
svc = SVC(probability=True, random_state=2023)
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.7792207792207793

In [11]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.7077922077922078

In [12]:
# Logisitc Regression
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train, y_train)
lrc.score(X_test, y_test)

0.8051948051948052

In [13]:
from sklearn.ensemble import VotingClassifier
voc = VotingClassifier(
    estimators=[('SVC', svc), ('KNN', knn), ('LRC', lrc)],
    voting='soft'
)
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.7857142857142857

In [18]:
from sklearn.model_selection import GridSearchCV

params = { 'SVC__C':[1.2, 1.3, 1.4], 'LRC__C':[4.8, 4.9, 5.0]}
grid_voc = GridSearchCV(voc, params, scoring='accuracy', cv=5)
grid_voc.fit(X_train, y_train)
grid_voc.best_params_

{'LRC__C': 4.9, 'SVC__C': 1.3}

In [19]:
grid_voc.best_estimator_.score(X_train, y_train)

0.8078175895765473

3. 결정트리를 기반으로 하는 앙상블 학습기 3종류를 이용하여, 정확도 / 정밀도 /  
   재현율을 구하세요.

In [54]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb.score(X_test, y_test)


0.7402597402597403

In [55]:
params = {'max_depth':[2, 3, 4, 5, 6]}
grid_xgb = GridSearchCV(xgb, params, scoring='accuracy', cv=5)
grid_xgb.fit(X_train, y_train)
grid_xgb.best_params_

{'max_depth': 3}

In [56]:
grid_xgb.best_estimator_.score(X_test, y_test)
pred_xgb = grid_xgb.best_estimator_.predict(X_test)

In [72]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
lgb.score(X_test, y_test)
pred_lgb = lgb.predict(X_test)

0.7272727272727273


In [58]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2023)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)
pred_rfc = rfc.predict(X_test)

- 평가

In [59]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

# 정확도 비교
ac_xgb = accuracy_score(y_test, pred_xgb)
ac_lgb = accuracy_score(y_test, pred_lgb)
ac_rfc = accuracy_score(y_test, pred_rfc)

In [61]:
# 정밀도 비교
ps_xgb = precision_score(y_test, pred_xgb)
ps_lgb = precision_score(y_test, pred_lgb)
ps_rfc = precision_score(y_test, pred_rfc)

In [62]:
# 재현율 비교
rs_xgb = recall_score(y_test, pred_xgb)
rs_lgb = recall_score(y_test, pred_lgb)
rs_rfc = recall_score(y_test, pred_rfc)

In [68]:
print('\txgboost\tlightgbm RandomForest')
print(f'정확도 : {ac_xgb:.4f}\t{ac_lgb:.4f}\t {ac_rfc:.4f}')
print(f'정밀도 : {ps_xgb:.4f}\t{ps_lgb:.4f}\t {ps_rfc:.4f}')
print(f'재현율 : {rs_xgb:.4f}\t{rs_lgb:.4f}\t {rs_rfc:.4f}')

	xgboost	lightgbm RandomForest
정확도 : 0.7338	0.7273	 0.7597
정밀도 : 0.6383	0.6200	 0.6735
재현율 : 0.5556	0.5741	 0.6111
